<a href="https://colab.research.google.com/github/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ĐỒ ÁN LẬP TRÌNH SONG SONG ỨNG DỤNG**

# Xoá background dựa trên Superpixel segmentation và Grabcut

## Giáo viên: Trần Trung Kiên

### Sinh viên:
- 1712568	KIM ĐÌNH LỘC	
- 1712787	NGUYỄN VĂN THÌN	
- 1712898	TRẦN VIỆT VĂN	


# 0. Ý NGHĨA TỪ NGỮ

**Superpixel** là kết quả của việc nhóm các pixel theo cảm nhận màu sắc, hoặc theo cách nhìn khác, đó là kết quả của sự phân đoạn hình ảnh.([Tu-chemnitz.de. 2022. Superpixel | TU Chemnitz](https://www.tu-chemnitz.de/etit/proaut/en/research/superpixel.html))


**Superpixel segmentation** là quá trình phân đoạn hình ảnh thành nhiều superpixels.

 <img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Dog_SLIC.jpg" 
     width="640" 
     height="455" />



GrabCut là phương pháp để phân đoạn một cách chính xác tiền cảnh của hình ảnh khỏi hậu cảnh.

#1 MÔ TẢ ỨNG DỤNG

**Ứng dụng mà nhóm chọn là Xoá background dựa trên ứng dụng segmentation của SLIC**.  
- Input: Một tấm ảnh RGB, mô hình yolov3
  
  <img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Dog.JPG" 
     width="640" 
     height="455" />

- Output: Ảnh đầu vào được remove background.  
<img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Dog_output.jpg" 
     width="640" 
     height="455" />


- Ý nghĩa của ứng dụng:
  - Hỗ trợ tách/lọc nền trong việc chỉnh sửa ảnh.
  - Nghiên cứu về segmentation việc xoá background.
  - Giảmđộ phức tạp của các bước xử lý ảnh tiếp theo.
- Lý do cần tăng tốc ứng dụng:
	- Ứng dụng sẽ chạy chậm nếu cài đặt tuần tự.
	- Nhu cầu xử lý dữ liệu lớn, ảnh có độ phân giải cao hay việc xử lý theo thời gian thực
- Tiềm năng song song hóa của ứng dụng: 
  - Song song hoá việc chuyển từ hệ màu RGB sang hệ màu CELAB
  - Thực hiện đồng thời quá trình khởi tạo các cụm của SLIC.
  - Thực hiện đồng thời quá trình tính từ trung tâm cụm đến từng pixel trong vùng.
  - Thực hiện song song hoá quá trình xử lý hình ảnh
- Thách thức:
  - Quá trình song song hoá việc kết nối các pixel trong SLIC có thể là một việc khó khăn.
  - Việc song song hoá mô hình GRABCUT của OpenCV rất khó khăn.
  - Thời gian xử lý lâu hơn so với model Machine Learning/deep learning.
  - Độ chính xác không cao so với deep learning.

#2 CÀI ĐẶT TUẦN TỰ

##2.1 Thiết kế mô hình

Ở đây chúng ta sẽ sử dụng mô hình 

<img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/HOST.JPG" 
     width="1152" 
     height="360" />

## 2.2 Thử nghiệm

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
!wget https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Dog.JPG
!wget https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/src/sequential.py
!wget https://upload.wikimedia.org/wikipedia/commons/0/05/Cow-bw.JPG

--2022-05-11 13:29:30--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  45.7MB/s    in 5.8s    

2022-05-11 13:29:36 (40.9 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]

--2022-05-11 13:29:36--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg.1’

yolov3.cfg.1        100%[===================>]   8.15K  --.-KB/s    in 0s 

In [ ]:
%%time
!python sequential.py /content/Dog.JPG

CPU times: user 1.86 s, sys: 268 ms, total: 2.13 s
Wall time: 4min 28s


#3 CÀI ĐẶT SONG SONG

## 3.1 Mô hình song song hoá dự kiến

<img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/DEVICE.JPG" 
     width="1152" 
     height="360" />

# TÀI LIỆU THAM KHẢO

Grabcut https://cvg.ethz.ch/teaching/cvl/2012/grabcut-siggraph04.pdf

Ảnh minh hoạ bởi: https://www.reddit.com/r/dogpictures/comments/6puk20/i_am_speed_because_i_am_speedy/

Achanta, Radhakrishna, et al. Slic superpixels. No. REP_WORK. 2010. [Chi tiết tại đây!](https://infoscience.epfl.ch/record/149300)

Le, Cuong Vo, et al. "Superpixel-based background removal for accuracy salience person re-identification." 2016 IEEE International Conference on Consumer Electronics-Asia (ICCE-Asia). IEEE, 2016. [Chi tiết tại đây!](https://ieeexplore.ieee.org/abstract/document/7804806/)

